# ライブラリの準備

Pytorchのライブラリ(lib)をzip形式でGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5<br>
取得したリンクからidを抜き出す。<br>
**1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5**

In [0]:
# Google Cloud SDKで、Google Driveのファイルを、
# Google Colabolatoryの一時領域にダウンロードするための認証
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Pytorchのライブラリをダウンロード
id = '1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
import os
os.chdir('/content')
downloaded.GetContentFile('lib.zip')

In [3]:
# ダウンロードを確認
!pwd
%ls

/content
adc.json  lib.zip  sample_data/


In [4]:
# ライブラリを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "lib"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall() # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
os.remove(save_path) # ZIPファイルを消去

CPU times: user 773 µs, sys: 2.97 ms, total: 3.74 ms
Wall time: 4.75 ms


In [5]:
# 処理結果を確認
!pwd
!ls
%ls ./lib/

/content
adc.json  lib  sample_data
architecture.py  dataloader.py  network_finetuning.py  optimizer.py  trainer.py
common.py        __init__.py    network.py             support.py


# データセットの準備

## データの格納／展開

データセット(zip)をローカルにダウンロードする。<br>
https://www.kaggle.com/daggysheep/find-a-car-park

ダウンロードしたデータセット(zip)をGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV<br>
取得したリンクからidを抜き出す。<br>
**17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV**

In [6]:
# データセットをダウンロード
%%time
id = '17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
os.chdir('/content')
downloaded.GetContentFile('find-a-car-park.zip')

CPU times: user 3.23 s, sys: 10.5 s, total: 13.8 s
Wall time: 1min 1s


In [7]:
# ダウンロードを確認
!pwd
%ls

/content
adc.json  find-a-car-park.zip  lib/  sample_data/


In [8]:
# データセットを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "find-a-car-park"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall(save_name) # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
#os.remove(save_path) # ZIPファイルを消去 # 試験中は消さない方が良い

CPU times: user 12.7 s, sys: 3.28 s, total: 16 s
Wall time: 56.2 s


In [9]:
# 処理結果を確認
%ls ./find-a-car-park/data

Free/  Full/


In [10]:
# データ個数を確認
!ls -1U "/content/find-a-car-park/data/Free" | wc -l
!ls -1U "/content/find-a-car-park/data/Full" | wc -l

1067
2195


## trainデータ／validationデータの分割

In [11]:
os.chdir('/content/find-a-car-park')
!pwd
%ls

/content/find-a-car-park
data/


In [12]:
# trainデータ、valデータを格納するフォルダを作成
os.mkdir("/content/find-a-car-park/train/")
os.mkdir("/content/find-a-car-park/val/")
!pwd
%ls

/content/find-a-car-park
data/  train/  val/


In [13]:
# フォルダが空であることを確認
!ls "/content/find-a-car-park/train/"
!ls -1U "/content/find-a-car-park/train/" | wc -l
!ls "/content/find-a-car-park/val/"
!ls -1U "/content/find-a-car-park/val/" | wc -l

0
0


In [14]:
#「/content/find-a-car-park/data/」以下の画像を「train」「val」フォルダに振り分ける
# train:val = 3:1
import glob
import shutil
os.chdir('/content/find-a-car-park/data')
label_dirs = os.listdir(path='.')
path = os.getcwd()  

# ラベル名が付与されている各フォルダごとに、中身のファイル名を取り出す
for dir_name in label_dirs:
  files = os.listdir(path + "/" + dir_name) # 1個のラベルフォルダのパスを取得
  numof_data = len(files) # ラベルフォルダ中のデータ数を取得
  files = glob.glob("./" + dir_name + "/*") # ラベルフォルダ中の全ファイルリストを取得

  # ラベルフォルダ中のファイルを抽出して、"val"フォルダに移動する
  # 全体の1/4のデータを"val"フォルダに移動するまで繰り返す
  i = 0
  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    i = i + 1
    if i < numof_data * 1/4:
      data_dir = "/content/find-a-car-park/val/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
    else:
      data_dir = "/content/find-a-car-park/train/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
  shutil.rmtree("./" + dir_name) # データを振り分け終えたラベルフォルダは削除しておく

Number of data in Free: 1067
Number of data in Full: 2195


In [15]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/train/Free" | wc -l
!ls -1U "/content/find-a-car-park/train/Full" | wc -l

801
1647


In [16]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/val/Free" | wc -l
!ls -1U "/content/find-a-car-park/val/Full" | wc -l

266
548


In [0]:
# [Option] データセットを一旦消す場合に実行
#%rm -r '/content/find-a-car-park'

## [DELETED] Google Drive上でデータセットを解凍して使用する場合

In [0]:
# Google Driveをmountする。
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park.zip'

In [0]:
!ls

In [0]:
!mkdir '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

In [0]:
!mv '/content/data' '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

# 学習／検証

In [17]:
%%time
os.chdir('/content/')
!npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.734s
     |████████████████████████████████| 686kB 5.1MB/s 
     |████████████████████████████████| 204kB 53.2MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=42b4e60304bb1d1f3e2426b9e4992c75687dc2341fa316158b2d313e4e805e98
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=fabbc81446a64e714515f2a523727d9f53725252cc5fe7a67950992b0efb0e9e
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 104 ms, sys: 158 ms, total: 262 ms
Wall time: 16.9 s


In [18]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 10.9 ms, sys: 28.2 ms, total: 39.1 ms
Wall time: 6.17 s


In [19]:
!pwd
!ls

/content
adc.json	 find-a-car-park.zip  sample_data  visdomlog.txt
find-a-car-park  lib		      url.txt


In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [21]:
# アーキテクチャを選択して使用する。
%%time
#net = network.VGG16(class_size=2)
#net = network.InceptionV3(class_size=2)
net = network.ResNet50(class_size=2)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
%%time
batch_size = 64
# フォルダのパスはかえる
train_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/train/', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/val/', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 70.1 ms, sys: 5.45 ms, total: 75.6 ms
Wall time: 78.6 ms


In [24]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.4 ms, sys: 39.9 ms, total: 42.2 ms
Wall time: 84.8 ms


In [25]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-25 03:40:21.725428
val Loss: 1.7324 Acc: 0.3268
----------
Epoch 2/50 2020-01-25 03:40:47.105565
train Loss: 1.0736 Acc: 0.8893
val Loss: 3.1792 Acc: 0.6732
----------
Epoch 3/50 2020-01-25 03:42:31.107541
train Loss: 2.5228 Acc: 0.5041
val Loss: 8.8473 Acc: 0.6732
----------
Epoch 4/50 2020-01-25 03:44:15.436657
train Loss: 1.9493 Acc: 0.3950
val Loss: 32.0084 Acc: 0.6732
----------
Epoch 5/50 2020-01-25 03:45:59.338369
train Loss: 0.8078 Acc: 0.6544
val Loss: 7.0116 Acc: 0.6732
----------
Epoch 6/50 2020-01-25 03:47:43.861340
train Loss: 1.7676 Acc: 0.6340
val Loss: 7.8369 Acc: 0.6732
----------
Epoch 7/50 2020-01-25 03:49:28.149929
train Loss: 2.8791 Acc: 0.3366
val Loss: 0.7783 Acc: 0.6732
----------
Epoch 8/50 2020-01-25 03:51:12.380480
train Loss: 0.6774 Acc: 0.6193
val Loss: 0.8736 Acc: 0.6732
----------
Epoch 9/50 2020-01-25 03:52:56.981475
train Loss: 0.8379 Acc: 0.6560
val Loss: 0.8557 Acc: 0.6732
----------
Epoch 10/50 2020-01-25 03:54:41.322004

In [26]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 67 %
CPU times: user 1.1 s, sys: 847 ms, total: 1.94 s
Wall time: 24.5 s


------------------